# VRDB Extract: Roll up to Parcel

In [15]:
import pandas as pd
import queries

In [16]:
pd.options.display.max_columns, pd.options.display.max_rows = 100,100

In [23]:
vrdb = pd.read_csv('202010_VRDB_Extract.txt', sep='|', encoding='latin')
vhist = pd.read_csv('2019-2020_VotingHistoryExtract.txt', sep='|', encoding='latin')
addpoint = pd.read_csv('address_point.csv')
b20a = pd.read_csv('ballots20a.csv', encoding='latin')
b20b = pd.read_csv('ballots20b.csv', encoding='latin')
pres = pd.concat([b20a,b20b])

In [25]:
addpoint['CTYNAME'].fillna(addpoint['POSTALCTYNAME'], inplace=True)
addpoint['compadd'] = addpoint['COMPRESS_ADDR'].fillna('') + addpoint['CTYNAME'].str.replace(' ','')
addpoint = addpoint.groupby('compadd').agg('first').reset_index()

-----

In [30]:
vhist['voted19'] = 1
voted19 = vhist.loc[vhist['ElectionDate'] == '2019-11-05',['StateVoterID','voted19']]
pres['voted20'] = 1
voted20 = pres[['Voter ID','voted20']]

In [31]:
vrdb = pd.merge(vrdb,voted19, how='left', left_on='StateVoterID', right_on='StateVoterID')
vrdb = pd.merge(vrdb,voted20, how='left', left_on='StateVoterID', right_on='Voter ID')

In [34]:
vrdb['compadd'] = (vrdb.RegStNum.astype(str).fillna('') + vrdb.RegStFrac.fillna('') + vrdb.RegStPreDirection.fillna('') + vrdb.RegStName.fillna('') + vrdb.RegStType.fillna('') + vrdb.RegStPostDirection.fillna('') + vrdb.RegCity.fillna('')).replace(r'\s+','',regex=True).replace('nan','')

In [35]:
kcv = vrdb[(vrdb['CountyCode'] == 'KI') & (vrdb['StatusCode'] == 'Active')].merge(addpoint, how='outer', left_on='compadd', right_on='compadd')

In [36]:
kcv['MAJOR'] = kcv['MAJOR'].fillna(0).astype(int).astype(str).str.zfill(6)
kcv.loc[~kcv['MINOR'].astype(str).str.isalpha() == True,'MINOR'] = kcv.loc[~kcv['MINOR'].astype(str).str.isalpha() == True,'MINOR'].astype(str).str.zfill(4)

In [37]:
kcv['voter_age'] = 2020 - kcv['Birthdate'].astype('datetime64[ns]').apply(lambda x : x.year)
kcv['voter_tenure'] = 2020 - kcv['Registrationdate'].astype('datetime64[ns]').apply(lambda x : x.year)
kcv['gender'] = kcv['Gender'].replace(['M','F','U','O'],[1,0,0,0])
kcv['voters'] = 1

In [38]:
kcv['gender'].fillna(0, inplace=True)
kcv['gender'] = kcv['gender'].astype(int)

In [39]:
parcel_voters = kcv.dropna(subset=['StateVoterID']).groupby(['MAJOR','MINOR']).agg({'RegZipCode' : 'first', 'voter_age' : 'mean','voter_tenure' : 'mean','gender' : 'mean','voters' : 'sum','voted19' : 'sum', 'voted20' : 'sum'}).reset_index()

In [40]:
parcel_voters.reset_index().to_pickle('voters')

In [41]:
from texter import send
send('Exported voters')